# Does the poker code run?

In [1]:
from poker import Card, Evaluator, Player, Game

In [2]:
g = Game(6, 100, 10000) 

In [11]:
%%time
g.play_one_complete_round()

The position of the small blind is 3
[2, 3]
The order is [2, 3, 4, 5, 0, 1]
1's hand is [('a', '♥'), ('6', '♦')] 
2's hand is [('5', '♥'), ('j', '♦')] 
3's hand is [('6', '♥'), ('6', '♣')] 
4's hand is [('q', '♣'), ('7', '♣')] 
5's hand is [('2', '♠'), ('8', '♦')] 
6's hand is [('j', '♣'), ('3', '♣')] 
Player 3: I have bet the small blind of $100!
Player 4: I have bet the big blind of $200!
Player 5: I am curious what is going to happen.            I am checking!
Player 5: I have 3 and am betting $5200!
Player 6: I have 5 and am betting $5400!
Player 1: I think I can win this round. I am calling!
Player 1: I have 4 and am betting $5400!
Player 2: I have 5 and am betting $5600!
The highest bid is 5600
Player 3: I have 5 and am betting $5800!
The highest bid is 5800
Player 4: I have 5 and am betting $6000!
The highest bid is 6000
Player 5: I have 5 and am betting $6200!
The highest bid is 6200
Player 6: I think I can win this round. I am calling!
Player 6: I have 4 and am betting $6200!
